In [ ]:
# Scraping data menggunakan API Twitter 
import tweepy as tw
import pandas as pd
 
# Menggunakan token dari twitter
consumer_key= '2kAfWmPvy5nPcoTOoRoIqSaIM'
consumer_secret= 'hkA4WAIWqS59s30QfMrj0EKJzHVf2bjBdif1HZM5DE6iUkaSX7'
access_token='1539625531591004163-IuSh4LkDgd9wXmyaPvuosVBKUslKv5'
access_token_secret='96bkZCGovhBLmLSVlu48Nlsj5xSp7rSQtvMwRggcopxLf'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth, wait_on_rate_limit=True)

# menentukan query
search_words = "aplikasi pln mobile"
new_search   = search_words + " -filter:retweets"

tweets = tw.Cursor(api.search, 
                   q                = new_search,
                   count            = 10000,
                   include_entities = True).items(10000)

users_locs = [[tweet.created_at,
               tweet.text] for tweet in tweets]

tweet_text = pd.DataFrame(data = users_locs, 
                          columns = ["Tanggal",
                                     "Teks"])

tweet_text.to_csv('plnmobile.csv', index=False)

In [2]:
# Import libraries and Read Dataset
import pandas as pd
df= pd.read_csv('/content/plnmbile.csv')
df.head()

,Tanggal,Tweet Id,Teks,Username
0,2022-06-30 21:29:11+00:00,1542621292285030400,@novitamuhayani 3. Pilih ID Pelanggan yang sud...,pln_123
1,2022-06-30 21:29:08+00:00,1542621281795063808,"@novitamuhayani Hi Kak, untuk informasi tsb Kk...",pln_123
2,2022-06-30 21:28:57+00:00,1542621235481563136,"@novitamuhayani Hi Kak, untuk informasi tsb Kk...",pln_123
3,2022-06-30 21:23:24+00:00,1542619836844748800,@dwifortuna34 2. Kakak juga dapat lapor melalu...,pln_123
4,2022-06-30 20:24:12+00:00,1542604938282340352,@pln_123 Di aplikasi sudah tercover kak tapi t...,YBudyyy


In [7]:
# Cek kolom dan baris

df.shape

(10206, 4)

In [6]:
# Cek Tanggal 

df.Tanggal.min(), df.Tanggal.max()

('2022-06-08 18:39:17+00:00', '2022-11-30 22:06:14+00:00')

In [8]:
# Cek tipe data
df.dtypes

Tanggal     object
Tweet Id     int64
Teks        object
Username    object
dtype: object

**Membuat Sentimen otomatis**

In [9]:
# Mengubah tipe data dari kolom tanggal
df['Tanggal'] = pd.to_datetime(df['Tanggal'], format='%Y-%m-%d')

In [10]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
ps = lambda x : sid.polarity_scores(x)
sentiment_scores = df.Teks.apply(ps)
sentiment_scores

0        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1        {'neg': 0.0, 'neu': 0.933, 'pos': 0.067, 'comp...
2        {'neg': 0.0, 'neu': 0.872, 'pos': 0.128, 'comp...
3        {'neg': 0.0, 'neu': 0.873, 'pos': 0.127, 'comp...
4        {'neg': 0.094, 'neu': 0.906, 'pos': 0.0, 'comp...
                               ...                        
10201    {'neg': 0.0, 'neu': 0.926, 'pos': 0.074, 'comp...
10202    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
10203    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
10204    {'neg': 0.0, 'neu': 0.924, 'pos': 0.076, 'comp...
10205    {'neg': 0.0, 'neu': 0.909, 'pos': 0.091, 'comp...
Name: Teks, Length: 10206, dtype: object

In [12]:
sentiment_df = pd.DataFrame(data = list(sentiment_scores))
sentiment_df.head()

,neg,neu,pos,compound
0,0.000,1.000,0.000,0.0000
1,0.000,0.933,0.067,0.3612
2,0.000,0.872,0.128,0.3612
3,0.000,0.873,0.127,0.4404
4,0.094,0.906,0.000,-0.4019


In [15]:
# Menentukan sentimen positif, negatif, dan netral
labelize = lambda x : 'Netral' if x==0 else('Positif' if x>0 else 'Negatif')
sentiment_df['sentimen'] = sentiment_df.compound.apply(labelize)
sentiment_df.head()

,neg,neu,pos,compound,sentimen
0,0.000,1.000,0.000,0.0000,Netral
1,0.000,0.933,0.067,0.3612,Positif
2,0.000,0.872,0.128,0.3612,Positif
3,0.000,0.873,0.127,0.4404,Positif
4,0.094,0.906,0.000,-0.4019,Negatif


In [16]:
# Jumlah masing-masing sentimen
sentiment_df['sentimen'].value_counts()

Netral     5158
Positif    4960
Negatif      88
Name: sentimen, dtype: int64

In [17]:
data = df.join(sentiment_df.sentimen)
data.head()

,Tanggal,Tweet Id,Teks,Username,sentimen
0,2022-06-30 21:29:11+00:00,1542621292285030400,@novitamuhayani 3. Pilih ID Pelanggan yang sud...,pln_123,Netral
1,2022-06-30 21:29:08+00:00,1542621281795063808,"@novitamuhayani Hi Kak, untuk informasi tsb Kk...",pln_123,Positif
2,2022-06-30 21:28:57+00:00,1542621235481563136,"@novitamuhayani Hi Kak, untuk informasi tsb Kk...",pln_123,Positif
3,2022-06-30 21:23:24+00:00,1542619836844748800,@dwifortuna34 2. Kakak juga dapat lapor melalu...,pln_123,Positif
4,2022-06-30 20:24:12+00:00,1542604938282340352,@pln_123 Di aplikasi sudah tercover kak tapi t...,YBudyyy,Negatif


In [20]:
# Memecah Sentimen per bulan dan hari

data['Bulan'] = pd.DatetimeIndex(data['Tanggal']).month
data['Hari'] = pd.DatetimeIndex(data['Tanggal']).day
data.head()

,Tanggal,Tweet Id,Teks,Username,sentimen,Bulan,Hari
0,2022-06-30 21:29:11+00:00,1542621292285030400,@novitamuhayani 3. Pilih ID Pelanggan yang sud...,pln_123,Netral,6,30
1,2022-06-30 21:29:08+00:00,1542621281795063808,"@novitamuhayani Hi Kak, untuk informasi tsb Kk...",pln_123,Positif,6,30
2,2022-06-30 21:28:57+00:00,1542621235481563136,"@novitamuhayani Hi Kak, untuk informasi tsb Kk...",pln_123,Positif,6,30
3,2022-06-30 21:23:24+00:00,1542619836844748800,@dwifortuna34 2. Kakak juga dapat lapor melalu...,pln_123,Positif,6,30
4,2022-06-30 20:24:12+00:00,1542604938282340352,@pln_123 Di aplikasi sudah tercover kak tapi t...,YBudyyy,Negatif,6,30


In [26]:
# Jumlah sentimen per bulannya (tidak memandang positif, negatif, atau netral)

bulan = data.groupby(data['Tanggal'].dt.strftime('%m'))['sentimen'].count()
bulan

Tanggal
06    1966
07    1436
08    1751
09    1651
10    1701
11    1701
Name: sentimen, dtype: int64

In [29]:
# Jumlah sentimen per bulan serta dalam hariannya (tidak memandang positif, negatif, atau netral)

bulan2 = data.groupby(['Bulan','Hari']).agg({'sentimen':'count'}).reset_index()
bulan2 = bulan2.rename(columns={'sentimen': 'jumlah'})
bulan2

,Bulan,Hari,jumlah
0,6,8,14
1,6,9,68
2,6,10,89
3,6,11,109
4,6,12,120
...,...,...,...
162,11,26,50
163,11,27,82
164,11,28,63
165,11,29,64


In [40]:
# Menentukan sentimen positif

sentimen_positif = data[data['sentimen'] == 'Positif']
sentimen_positif.shape

(4960, 7)

In [41]:
# Menentukan sentimen netral

sentimen_netral = data[data['sentimen'] == 'Netral']
sentimen_netral.shape

(5158, 7)

In [42]:
# Menentukan sentimen negatif

sentimen_negatif = data[data['sentimen'] == 'Negatif']
sentimen_negatif.shape

(88, 7)

In [43]:
# Jumlah sentimen per bulan serta dalam hariannya (sentimen positif)

sentimen_positif_per_bulan = sentimen_positif.groupby(['Bulan','Hari']).agg({'sentimen':'count'}).reset_index()
sentimen_positif_per_bulan = sentimen_positif_per_bulan.rename(columns={'sentimen': 'jumlah'})
sentimen_positif_per_bulan

,Bulan,Hari,jumlah
0,6,9,23
1,6,10,43
2,6,11,32
3,6,12,33
4,6,13,15
...,...,...,...
161,11,26,26
162,11,27,24
163,11,28,25
164,11,29,36


In [44]:
# Jumlah sentimen per bulan serta dalam hariannya (sentimen netral)

sentimen_netral_per_bulan = sentimen_netral.groupby(['Bulan','Hari']).agg({'sentimen':'count'}).reset_index()
sentimen_netral_per_bulan = sentimen_netral_per_bulan.rename(columns={'sentimen': 'jumlah'})
sentimen_netral_per_bulan

,Bulan,Hari,jumlah
0,6,8,14
1,6,9,45
2,6,10,46
3,6,11,77
4,6,12,87
...,...,...,...
162,11,26,24
163,11,27,58
164,11,28,38
165,11,29,28


In [45]:
# Jumlah sentimen per bulan serta dalam hariannya (sentimen negatif)

sentimen_negatif_per_bulan = sentimen_negatif.groupby(['Bulan','Hari']).agg({'sentimen':'count'}).reset_index()
sentimen_negatif_per_bulan = sentimen_negatif_per_bulan.rename(columns={'sentimen': 'jumlah'})
sentimen_negatif_per_bulan

,Bulan,Hari,jumlah
0,6,14,2
1,6,17,2
2,6,21,4
3,6,22,1
4,6,23,1
5,6,30,4
6,7,3,1
7,7,4,1
8,7,5,3
9,7,6,1


**Teks Pra-pemrosesan**

In [46]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize # Tokenization
nltk.download('stopwords')
from nltk.corpus import stopwords # Stopwords

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer # Lemmatization


df_text = data['Teks'].astype(str)
lines = df_text.values.tolist()
list_tokens_lemma = list()
lemmatizer = WordNetLemmatizer()

for line in lines: # Normalization
  line = line.replace("[\#\@\!]\w+","")
  line = line.replace("http\S+","")
  line = line.replace("http\S+|www\S+|https\S+","")
  line = line.replace("[\d+]","")
  line = line.replace("RT[\s]+","")
  line = line.replace("[\.\,\;\!\?\*\$\"\'\:]","")
  tokens = word_tokenize(line)
  # merubah menjadi huruf kecil semua
  tokens = [w.lower() for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  # menghapus stopwords
  tokens = [word for word in tokens if not word in stopwords.words()]
  # melakukan lemmatization
  tokens = [lemmatizer.lemmatize(word) for word in tokens]

  list_tokens_lemma.append(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


KeyboardInterrupt: ignored

In [ ]:
print(lines[0])
print(list_tokens_lemma[0])

@kepoludora Hi Kak, baik Admin respon terkait DM nya yah. Agar semua makin mudah, ayo download aplikasi New PLN Mobile di Play Store atau Apps Store, Trims-Dewita
['kepoludora', 'admin', 'respon', 'terkait', 'dm', 'nya', 'mudah', 'ayo', 'download', 'aplikasi', 'pln', 'mobile', 'play', 'store', 'apps', 'store']


**Feature Extraction**

**TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
new_doc = list()

for doc in list_tokens_lemma:
  row = ' '.join(doc)
  new_doc.append(row)

tfidf_vectorizer = TfidfVectorizer()
X_input_tfidf = tfidf_vectorizer.fit_transform(new_doc)

In [ ]:
print(tfidf_vectorizer.get_feature_names())
print(X_input_tfidf.toarray())
print(tfidf_vectorizer.vocabulary_)
print(X_input_tfidf.shape)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aaadisty', 'aaaknid', 'aadbitcoin', 'aadonon', 'aajieputra', 'aalanferari', 'aalhenna', 'aanbayuerr', 'aandryans', 'aanjabii', 'aapabila', 'aardenerni', 'aauuuull', 'abadi', 'abadilaia', 'abadipb', 'abaggeur', 'abarlowdickz', 'abcditaf', 'abcdonghyucke', 'abcunaaaa', 'abdalloh', 'abduh', 'abduhbiano', 'abdul', 'abdulazisid', 'abdurrohimfajar', 'abet', 'abeteamone', 'abetsilitonga', 'abi', 'abidusan', 'abimanyudaniel', 'abintoro', 'abirizalj', 'abis', 'ablenkofficial', 'abraham', 'absltuke', 'abu', 'abuhaqqioke', 'abvghtf', 'abyandanusa', 'aca', 'acalsmart', 'acara', 'acces', 'account', 'acecuuu', 'aceh', 'acep', 'acepsaepulbahri', 'acepteatea', 'achibid', 'achietmokoginta', 'achmad', 'achmadzehn', 'achmed', 'acho', 'acilebut', 'acmeiraa', 'aconkrenzi', 'acupofcooffee', 'adadoaibu', 'adalaha', 'adamalamb', 'adambrotowijono', 'adamfrds', 'adamnovtadw', 'adapromo', 'adchaminoerdin', 'addogerardo', 'ade', 'adeenametanoia', 'adegragerudiant', 'adeharfiansyah', 'adeknyasojeong', 'adelia', 

**Pemodelan menggunakan Decision Tree**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from imblearn.metrics import sensitivity_specificity_support
from sklearn.tree import DecisionTreeClassifier


# tentukan X and y
df_X = X_input_tfidf
df_y = data['sentimen']

# Dibagi menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3)

# label encoding untuk y
le = LabelEncoder()
le.fit(y_train)
y_train=le.transform(y_train)
y_test=le.fit_transform(y_test)

#mulai learning 
model=DecisionTreeClassifier(criterion='entropy')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('Accuracy ',accuracy_score(y_test, y_pred))    
print('Precision ',precision_score(y_test, y_pred, average='weighted'))  
print('Recall ',recall_score(y_test, y_pred, average='weighted'))

Accuracy  0.9533488837054315
Precision  0.9544408663649565
Recall  0.9533488837054315
